# **Kết nối với Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# **Khai báo thư viên**

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
from keras.preprocessing import  image
from keras.preprocessing.image import load_img, img_to_array,array_to_img,ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D, Activation, AveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import callbacks
from tensorflow.keras.models import model_from_json

# **Cài đặt thư viện GDown, download và giải nén file data được update lên Google Drive**
Link Google Drive: https://drive.google.com/uc?id=1Ww5Uu9wC2ySTUYFaKEOjUUjm9zI7-zHA

In [ ]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1Ww5Uu9wC2ySTUYFaKEOjUUjm9zI7-zHA
!unzip Data_256.zip

# **Hàm dừng training khi đạt độ chính xác yêu cầu**

In [ ]:
ACCURACY_THRESHOLD = 0.988
class myCallback(callbacks.Callback):
	def on_epoch_end(self, epoch, logs={}):
		if((logs.get('val_accuracy') >= ACCURACY_THRESHOLD)):
			self.model.stop_training = True
e_callback = myCallback()

# **Load data**

In [ ]:
size = 256
x = []
y = []
for i in range(0,16):
  dir_folder = "/content/"+str(i+1)+'/'
  for j in os.listdir(dir_folder):
      img = image.load_img(dir_folder + j, color_mode = 'grayscale', target_size=(size,size))
      img = img_to_array(img)
      img = img.reshape(size,size,1)
      img = img.astype('float32')/255.0
      x.append(img)
      y.append(int(i))
x = np.array(x)
y = np.array(y)
x_train, x_val_test, y_train, y_val_test = train_test_split(x,y, test_size=0.2)
x_val, x_test, y_val, y_test = train_test_split(x_val_test,y_val_test, test_size=0.25)
y_train = np_utils.to_categorical(y_train,16)
y_val = np_utils.to_categorical(y_val,16)
y_test = np_utils.to_categorical(y_test,16)
x = []
y = []

# **Khai báo các lớp của Model CNN**

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = (3,3), strides = 1, padding = "same", activation = 'relu', input_shape = (size,size,1)))
model.add(Conv2D(filters = 16, kernel_size=(3,3), strides = 1, padding = "same",activation="relu"))
model.add(AveragePooling2D(pool_size = (2,2), strides = 2))

model.add(Conv2D(filters = 16, kernel_size=(3,3), strides = 1, padding = "same",activation="relu"))
model.add(AveragePooling2D(pool_size = (2,2), strides = 2))

model.add(Conv2D(filters = 16, kernel_size=(3,3), strides = 1, padding = "same",activation="relu"))
model.add(AveragePooling2D(pool_size = (2,2), strides = 2))

model.add(Conv2D(filters = 32, kernel_size=(3,3), strides = 1, padding = "same",activation="relu"))
model.add(AveragePooling2D(pool_size = (2,2), strides = 2))

model.add(Conv2D(filters = 32, kernel_size=(3,3), strides = 1, padding = "same",activation="relu"))
model.add(AveragePooling2D(pool_size = (2,2), strides = 2))

model.add(Conv2D(filters = 32, kernel_size=(3,3), strides = 1, padding = "same",activation="relu"))
model.add(AveragePooling2D(pool_size = (2,2), strides = 2))


model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation = 'softmax'))
model.summary()
opt = Adam(learning_rate=0.0008)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs = 1000, batch_size = 32, validation_data = (x_val, y_val), callbacks=[e_callback])

# **Tiến hành training Model**

In [ ]:
opt = Adam(learning_rate=0.0008)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs = 1000, batch_size = 32, validation_data = (x_val, y_val), callbacks=[e_callback])

# **Lưu file Model vào Google Drive**

In [ ]:
model_json = model.to_json()
with open("/content/drive/MyDrive/RecognizeWood_256.json", "w") as json_file:
      json_file.write(model_json)
model.save_weights("/content/drive/MyDrive/RecognizeWood_256.h5")
print("Saved model to disk")

Saved model to disk


# **Xuất đồ thị độ chính xác**

In [ ]:
plt.figure(figsize=(8, 6), dpi=200)
plt.grid(True)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.xlabel('Epoch',fontsize=16)
plt.xlim()
plt.legend(['Train','Validation'],loc='upper_left',fontsize=16)
plt.savefig('/content/drive/MyDrive/Acc_train.jpg', dpi = 200)
plt.show()

# **Xuất đồ thị độ mất mát**

In [ ]:
plt.figure(figsize=(8, 6), dpi=200)
plt.grid(True)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.xlabel('Epoch',fontsize=16)
plt.xlim()
plt.legend(['Train','Validation'],loc='upper_left',fontsize=16)
plt.savefig('/content/drive/MyDrive/Loss_train.jpg', dpi = 200)
plt.show()

# **Load Model đã save từ Google Drive**

In [ ]:
model_architecture = "/content/drive/MyDrive/RecognizeWood_256.json"
model_weights = "/content/drive/MyDrive/RecognizeWood_256.h5"
model = model_from_json(open(model_architecture).read())
model.load_weights(model_weights)
opt = Adam(learning_rate=0.0008)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

# **Test độ chính xác của Model với dữ liệu không được đưa vào Training và Validation**

In [ ]:
score = model.evaluate(x_test,y_test,batch_size = 1)
print("Accuracy test:",score[1])
print("Loss test:",score[0])

# **Confution Matrix**

In [ ]:
labels = ["Anh đào","Bạch đằng","Bồ đề","Căm xe","Cao su","Cate","Hoàng đàn","Keo lai","Lát hoa","Mỡ ","Mun","Muồng đen","Quế","Sồi","Táu mật","Không phải gỗ"]
y_pred = model.predict(x_test)
y_true = y_test
y_pred=np.argmax(y_pred, axis=1)
y_true=np.argmax(y_true, axis=1)
cm = confusion_matrix(y_true, y_pred)
cm_normal = np.round(cm/np.sum(cm,axis=1).reshape(-1,1),2)
df_cm = pd.DataFrame(cm_normal)
plt.figure(figsize=(16, 12), dpi=160)
sns.heatmap(df_cm, cmap ="Greens", annot=True, xticklabels=labels, yticklabels=labels)
plt.title('Confusion matrix',fontsize=20)
plt.xlabel("Predict",fontsize=16)
plt.ylabel("Actual",fontsize=16)
plt.savefig('/content/drive/MyDrive/Confution_matrix.jpg', dpi = 160)